In [3]:
from researcher import Researcher
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [4]:
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8041"
# ADMIN_URL = "http://0.0.0.0:8041"

agent_config = {
    "webhook_host": WEBHOOK_HOST,
    "webhook_port": WEBHOOK_PORT,
    "webhook_base": WEBHOOK_BASE,
    "admin_url": ADMIN_URL
}



In [5]:
import torch

# models
from torch import nn
from torch import optim
from torch.autograd import Variable
import os
import sys

from opacus import PrivacyEngine

# The Researcher generates the model

model = nn.Sequential(
            nn.Linear(8, 4),
            nn.Sigmoid(),
            nn.Linear(4, 2),
            nn.Sigmoid(),
            nn.Linear(2, 1),
            nn.Sigmoid()
)

In [6]:
validation_data_path = 'data/coordinator_validation_data.csv'

In [7]:
researcher = Researcher(agent_config, validation_data_path, model)

CLEANING THE VALIDATION SET
VALIDATION SET HAS BEEN CLEANED


## Request Authorised Research Credential

Copy invite from the [Health Research Authority Notebook](http://localhost:8889/notebooks/Health%20Research%20Regulator.ipynb)

In [8]:
invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'a96950af-88b8-49a6-be0a-35f8b808caab', 'serviceEndpoint': 'http://172.17.0.1:8030', 'label': 'NHS Health Research Authority', 'recipientKeys': ['5JEH56xJaB9DGQ2DNTBDMjmvFqV58vKnU1p2WdFndwo1']}

In [9]:
await researcher.agent_controller.connections.accept_connection(invite)

Connection Handler Called
Connection d6907cc9-a7f3-4bcb-99f9-9304a11ff840 in State invitation
Connection Handler Called
Connection d6907cc9-a7f3-4bcb-99f9-9304a11ff840 in State request


{'invitation_key': '5JEH56xJaB9DGQ2DNTBDMjmvFqV58vKnU1p2WdFndwo1',
 'updated_at': '2021-02-19 14:47:19.987939Z',
 'my_did': 'GPuks4W4pKmqa7p73hiXNC',
 'created_at': '2021-02-19 14:47:19.970611Z',
 'connection_id': 'd6907cc9-a7f3-4bcb-99f9-9304a11ff840',
 'state': 'request',
 'their_label': 'NHS Health Research Authority',
 'initiator': 'external',
 'routing_state': 'none',
 'invitation_mode': 'once',
 'accept': 'manual',
 'request_id': '7c41742f-7a10-44e8-a278-ad9415678c9c'}

Connection Handler Called
Connection d6907cc9-a7f3-4bcb-99f9-9304a11ff840 in State response
Connection Handler Called
Connection d6907cc9-a7f3-4bcb-99f9-9304a11ff840 in State active
Handle Credentials
Credential exchange bf99cdf0-f5f7-46e3-b77a-19cb1f199cf1, role: holder, state: offer_received
Offering: [{'name': 'scope', 'value': 'diabetes'}]
Requesting credential
Handle Credentials
Credential exchange bf99cdf0-f5f7-46e3-b77a-19cb1f199cf1, role: holder, state: request_sent
Handle Credentials
Credential exchange bf99cdf0-f5f7-46e3-b77a-19cb1f199cf1, role: holder, state: credential_received
Storing Credential
Handle Credentials
Credential exchange bf99cdf0-f5f7-46e3-b77a-19cb1f199cf1, role: holder, state: credential_acked


## Configure Authentication Policy

Note: Normally we would specify the credential definition here specifying which entities we trust to issue this schema, but this changes every time because it's a demo.

Feel free to copy yours across from the NHS Trust though.

In [10]:


schema_id = "BvRKFu1fdzGdJzQcKb8e8p:2:NHS Hosptial:0.0.1"
# cred_def = "WfntKNFwXMQfgmU9ofbxPM:3:CL:156569:default"

revocation = False
exchange_tracing = False

#Enable this to ask for attributes to identity a user
#TODO - change restriction to schemaId or credentialId

req_attrs = [
    {"name": "name", "restrictions": [{"schema_id": schema_id}]},#, "cred_def_id": cred_def}]},
]

global indy_proof_request
indy_proof_request = {
    "name": "Proof of Hospital",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
}



In [11]:
researcher.set_authentication_policy(indy_proof_request)

## Send Invite to Hospital 1

Option: We could make this a multiuse invite.

In [12]:
researcher.new_dataowner_invite()

Connection Handler Called
Connection 9ae6ede7-4132-4c0d-925f-d34958e62c2d in State invitation

♫♫♫ > STEP 1: Send the aries invitation to the data owner

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "696b6287-63d5-49a0-8dea-2657fee2d826", "label": "Health Researcher", "recipientKeys": ["BU36Z9DJcNcT2XopUXgK29VF4y43xy4t4MQsPXbsuGyH"], "serviceEndpoint": "http://172.17.0.1:8040"}

Connection Handler Called
Connection 9ae6ede7-4132-4c0d-925f-d34958e62c2d in State request
Accepting request  9ae6ede7-4132-4c0d-925f-d34958e62c2d
Connection Handler Called
Connection 9ae6ede7-4132-4c0d-925f-d34958e62c2d in State response
Sending trust ping 9ae6ede7-4132-4c0d-925f-d34958e62c2d
Connection Handler Called
Connection 9ae6ede7-4132-4c0d-925f-d34958e62c2d in State active
Pending dataowner connection moved to active. Challenging with auth policy if set
Handle present proof
Role verifier, Exchange d379a7a1-d051-446e-9ed1-f93ccdb30319 in state request_sent
Handle pr

## Send Invite to Hospital 2

In [13]:
researcher.new_dataowner_invite()

Connection Handler Called
Connection 2fdd0e61-0b94-428d-bff7-f18f5dac8230 in State invitation

♫♫♫ > STEP 1: Send the aries invitation to the data owner

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "af4ca1f7-7070-42cc-943d-e7af24bef5c4", "label": "Health Researcher", "recipientKeys": ["DLbK5vsixEqenNevKHRxgK3jMZHApHHQZ12ix5FsM15Y"], "serviceEndpoint": "http://172.17.0.1:8040"}

Connection Handler Called
Connection 2fdd0e61-0b94-428d-bff7-f18f5dac8230 in State request
Accepting request  2fdd0e61-0b94-428d-bff7-f18f5dac8230
Connection Handler Called
Connection 2fdd0e61-0b94-428d-bff7-f18f5dac8230 in State response
Sending trust ping 2fdd0e61-0b94-428d-bff7-f18f5dac8230
Connection Handler Called
Connection 2fdd0e61-0b94-428d-bff7-f18f5dac8230 in State active
Pending dataowner connection moved to active. Challenging with auth policy if set
Handle present proof
Role verifier, Exchange 464cb34f-1daa-4107-a9d2-9ed7af092919 in state request_sent
Handle pr

## Send Invite to Hospital 3

In [ ]:
researcher.new_dataowner_invite()

## Initiate Learning

PPML Learning messages are sent over DIDComm using the basic message protocol. You will be able to see the encrypted message in the terminal you initialised this project.

In [14]:
researcher.initiate_learning()

MODEL OPENED FOR TRANSPORT
Training model with 2 DataOwners
Still learning
Continue Learning 2fdd0e61-0b94-428d-bff7-f18f5dac8230
HOSPITAL MODEL LOADED

PRINTING PARAMETERS:


Sequential(
  (0): Linear(in_features=8, out_features=4, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=2, out_features=1, bias=True)
  (5): Sigmoid()
)


0.weight tensor([[-0.0407, -2.7445, -0.5781,  1.4727,  0.9153,  0.9233,  2.1776,  0.1872],
        [ 1.3477, -0.8946,  1.9262, -1.6329, -0.3836, -0.7956, -1.6785,  0.8979],
        [-2.0327,  0.7583, -0.4663, -0.1193, -0.0782, -0.6535, -0.0044,  3.8578],
        [-2.2942, -0.9636, -1.6302, -0.8692, -0.2579, -0.3502,  0.5492,  0.5564]])
0.bias tensor([-2.0306, -1.5065, -2.8781,  2.0966])
2.weight tensor([[-2.3666, -2.4208, -3.4018,  3.0841],
        [ 1.9018,  2.4939,  2.5090, -2.9655]])
2.bias tensor([ 2.3059, -1.4538])
4.weight tensor([[-6.8204,  5.3866]])
4.bias tensor([-0.4698])
